In [1]:
import numpy as np
import pandas as pd

# create example data
data = pd.read_csv("rainfall_data.csv")
data=data.iloc[:,3:]
# calculate 95th percentile for each column
percentiles = data.quantile(0.95)

# find rows with values above the 95th percentile
above_95th = data[(data > percentiles).all(axis=1)]

print(above_95th)

       rainfall
6     24.932436
7     23.246956
41    33.846075
64    23.561225
83    22.738915
...         ...
2534  21.900691
2538  24.230811
2541  22.366934
2551  34.073259
2552  21.649361

[129 rows x 1 columns]


In [2]:
import pandas as pd
data = pd.read_csv("rainfall_data.csv")
data=data.iloc[:,3:]
# calculate 95th percentile for each column
percentiles = data.quantile(0.95)

# filter rows above 95th percentile for each column
filtered_data = data[(data > percentiles).any(axis=1)]




print(filtered_data)

       rainfall
6     24.932436
7     23.246956
41    33.846075
64    23.561225
83    22.738915
...         ...
2534  21.900691
2538  24.230811
2541  22.366934
2551  34.073259
2552  21.649361

[129 rows x 1 columns]


In [3]:
data = pd.read_csv("rainfall_data.csv")
# label extreme rows in original dataframe
data['rainfall_class'] = 'normal'
data.loc[data.index.isin(filtered_data.index), 'rainfall_class'] = 'extreme'


In [5]:
data

,date,windspeed,tpw,rainfall,rainfall_class
0,6/1/2000,12.245595,0.033285,4.688589,normal
1,6/2/2000,12.825491,0.044874,11.330589,normal
2,6/3/2000,12.921664,0.010243,6.894713,normal
3,6/4/2000,15.149001,0.036881,4.613324,normal
4,6/5/2000,18.495907,0.139491,8.111635,normal
...,...,...,...,...,...
2557,9/26/2020,5.577215,0.009963,4.467977,normal
2558,9/27/2020,5.184293,0.002341,1.572182,normal
2559,9/28/2020,4.469007,0.000867,1.819019,normal
2560,9/29/2020,4.259090,0.001416,2.198017,normal


In [6]:
import numpy as np
import copy
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras import backend as K
import os
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import seed
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc



In [7]:
extreme_data=data[data["rainfall_class"]=="extreme"]
normal_data=data[data["rainfall_class"]=="normal"]

In [53]:

def weighted_loss(normal_weight=10, extreme_weight=1):
    def loss(y_true, y_pred):
        normal_loss = K.mean(K.square(y_true[:, :3] - y_pred[:, :3]) * K.cast(y_true[:, 2] < 20.807473, 'float32')) * normal_weight
        extreme_loss = K.mean(K.square(y_true[:, :3] - y_pred[:, :3]) * K.cast(y_true[:, 2] >= 20.807473, 'float32')) * extreme_weight

        return (normal_loss + extreme_loss)
    return loss



In [55]:
train_data[:,:3]

array([[1.2245595e+01, 3.3285443e-02, 4.6885886e+00],
       [1.2825491e+01, 4.4874474e-02, 1.1330589e+01],
       [1.2921664e+01, 1.0243150e-02, 6.8947134e+00],
       ...,
       [4.4690070e+00, 8.6651702e-04, 1.8190190e+00],
       [4.2590899e+00, 1.4163180e-03, 2.1980174e+00],
       [5.5138383e+00, 2.4743171e-03, 1.2713634e+01]], dtype=float32)

In [54]:
reg_strengths = [0.1, 1, 10, 100]
num_epochs = 50
batch_size = 38
#train_data = normal_data[["windspeed_discretized","tpw_discretized"]].values.astype("float32")
train_data = normal_data[["windspeed","tpw","rainfall"]].values.astype("float32")

#best_reg_strength = find_best_reg_strength(train_data, reg_strengths, num_epochs, batch_size)

# Define the autoencoder architecture with the best regularization strength
input_layer = Input(shape=(3,),name='input')
encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=0.01),name='encoder')(input_layer)
decoded = Dense(3, activation=None, name='decoder')(encoded)

# Create the autoencoder
autoencoder = Model(input_layer, decoded)

# Compile the autoencoder with the weighted loss function
#autoencoder.compile(optimizer='adam', loss="mse")
autoencoder.compile(optimizer='adam', loss=weighted_loss())

# Train the model with the majority class
history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)
#history = autoencoder.fit(train_data,epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=0)


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'loss/mul' defined at (most recent call last):
    File "C:\Users\shine\anaconda3\envs\environment\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\shine\anaconda3\envs\environment\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\shine\anaconda3\envs\environment\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\shine\anaconda3\envs\environment\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\shine\anaconda3\envs\environment\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\shine\anaconda3\envs\environment\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\shine\a

In [25]:
from tensorflow.keras.models import load_model

# Save the model to a file
autoencoder.save("my_autoencoder_model.h5")


In [30]:
#test_data = data[["windspeed_discretized","tpw_discretized","rainfall_discretized"]].values
test_data = data[["windspeed","tpw","rainfall"]].values
autoencoder = load_model("my_autoencoder_model.h5", custom_objects={"custom_cost_sensitive_loss": custom_cost_sensitive_loss})

predicted_data = autoencoder.predict(test_data)

ValueError: Unknown loss function: cost_sensitive_loss. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [36]:
train_data.shape

(2433, 3)